In [5]:
file1 = open('Table6.txt', 'r')


In [2]:
Lines = file1.readlines()

NameError: name 'file1' is not defined

In [27]:
Lines = file1.readlines(3377031400)

In [1]:
print(len(Lines))

NameError: name 'Lines' is not defined

In [3]:
import pymysql.cursors
import time
import pandas as a

In [61]:
cols = Lines[3].split(",",12)
print(len(cols))
print(cols)
print(int(float(cols[4])))

13
['50100003', '3', '5', '26608033', '12.1', 'Equal or more than 3 surface-names', 'Wrong', '11', '26611282', '0', 'NA', '1', '26611282\n']
12


In [130]:
for i in range(0,115):
    print(Lines[i])


Entity_Mention_index, Source_Entity_Index, Dest_Entity_Index, Surface_Name_Entity_Index, Sentence_ID, Condition(Less/More), Correct SN/Wrong SN, Edit-Distance, Recommended Surface-Name, Yes/No Superstring, Recommended Superstring, Yes/No Substring, Recommended Substring

50100001,1,2,26608031,sentence_id,Lesser-than-3-surface-names

50100002,3,4,26608032,12.1,Equal or more than 3 surface-names,Correct

50100003,3,5,26608033,12.1,Equal or more than 3 surface-names,Wrong,11,26611282,0,NA,1,26611282

50100004,3,6,26608034,12.1,Equal or more than 3 surface-names,Correct

50100005,3,7,26608035,12.1,Equal or more than 3 surface-names,Correct

50100006,3,8,26608036,12.1,Equal or more than 3 surface-names,Correct

50100007,3,9,26608037,12.1,Equal or more than 3 surface-names,Wrong,3,26727615,0,NA,0,NA

50100008,3,10,26608038,12.1,Equal or more than 3 surface-names,Correct

50100009,3,11,26608039,12.2,Equal or more than 3 surface-names,Correct

50100010,3,12,26608040,12.2,Equal or more than 3 s

In [158]:

for i_og in range(1,41832473):
    if i_og % 500 == 0:
        start_t = time.time()
    if i_og % 500 == 499:
        end_t = time.time()
        string2= str((int(i_og/41832473*100)))+"% i.e. " +str(i_og) + " in "+str(end_t-start_t)

    line = Lines[i_og]
    cols = line.split(",",12)
    if len(cols)<13: 
        continue
    #print(cols)
    req_sen = cols[4]
    sql2_fetch = "Select id from sentences where id = " + str(cols[4])
    connection = pymysql.connect(host='localhost',port=int(3306),user='root',passwd='wikiwiki2',db='Wiki',charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
    with connection:
        with connection.cursor() as cursor:
                cursor.execute(sql2_fetch)
                id_check = cursor.fetchone()
        connection.commit()
    if id_check:
        continue
    sql2_fetch = "Select entityName from Entity where entityId = " + str(cols[1])
    connection = pymysql.connect(host='localhost',port=int(3306),user='root',passwd='wikiwiki2',db='Wiki',charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
    with connection:
        with connection.cursor() as cursor:
                cursor.execute(sql2_fetch)
                entity_name = cursor.fetchone()
        connection.commit()
    #print(entity_name['entityName'])
    sql2_fetch = "Select surfaceName from surfaceNames where surfaceNamesId = " + str(cols[3])
    connection = pymysql.connect(host='localhost',port=int(3306),user='root',passwd='wikiwiki2',db='Wiki',charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
    with connection:
        with connection.cursor() as cursor:
                cursor.execute(sql2_fetch)
                surface_used = cursor.fetchone()
        connection.commit()
    if (not entity_name) or (not surface_used):
        continue
    re = 0
    ch = req_sen[0]
    req_line = 0
    req_id = 0
    while ch != '.':
        req_id *= 10
        req_id += int(ch)
        re += 1
        ch = req_sen[re]
    re += 1
    ch = req_sen[re]
    while re < len(req_sen):
        req_line *= 10
        req_line += int(ch)
        re += 1
        if re < len(req_sen):
            ch = req_sen[re]
    #print(req_id)
    #print(req_line)
    url2= 'https://en.wikipedia.org/w/api.php?action=query&prop=extracts&explaintext&format=json&exsectionformat=wiki&titles='
    url2 = url2 + entity_name['entityName']
    r = requests.get(url=url2)
    k = r.json()
    #print(k)
    ins_flag = 0
    for x in k['query']['pages']:
        #print(x)
        if int(x) != req_id :
            continue
        #print(x)
        #print(k['query']['pages'])
        if not k['query']['pages'][x]['extract'] or not surface_used:
            continue
        i = k['query']['pages'][x]['extract'].lower().find(surface_used['surfaceName'].lower())
        #print(i)
        if i >= len(k['query']['pages'][x]['extract']) or i == -1:
            continue
        j = i
        ch = k['query']['pages'][x]['extract'][i]
        while ch != '.' and i>=0 :
            i -= 1
            ch = k['query']['pages'][x]['extract'][i]
        start = i +1
        ch = k['query']['pages'][x]['extract'][j]
        while ch != '.' and j< len(k['query']['pages'][x]['extract'])-1: 
            j+= 1
            ch = k['query']['pages'][x]['extract'][j]
        st = ""
        while start < j:
            st +=  k['query']['pages'][x]['extract'][start]
            start+= 1
        #print(st)
        
        sql2_push = "Insert ignore into sentences values(%s, %s);"
        connection = pymysql.connect(host='localhost',port=int(3306),user='root',passwd='wikiwiki2',db='Wiki',charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
        with connection:
            with connection.cursor() as cursor:
                cursor.execute(sql2_push,(cols[4],st))
            connection.commit()
    print(string2 ,end = "\r")

ConnectionError: ('Connection aborted.', TimeoutError(110, 'Connection timed out'))

In [157]:
i_og


2932247

In [4]:
i = 0
while i < 40797531:
    j = 0
    sql2 = "INSERT IGNORE INTO `Mention` (instanceId,dest_eid,mention_sid,source_eid,sentence,link,editDistance,isCorrectSubString,isCorrectSuperString,editSuggest,subSuggest,superSuggest) VALUES "
    start = time.time()
    while j < 500:
        line = Lines[i]
        cols = line.split(",",12)
       
        if len(cols)<13:
            j = j + 1 
            i = i + 1
            continue
        
        cols[12]= cols[12].replace('\n', '')
        for k in range(4,13):
            if cols[k] == "NA":
                cols[k] = str(-1)
        
        sql2 = ''.join([sql2,'("',cols[0],'","',cols[2],'","',cols[3],'","',cols[1],'","',cols[4],'","',cols[5],'","',cols[7],'","',cols[9],'","',cols[11],'","',cols[8],'","',cols[10],'","',cols[12],'"),'])
        
        i = i + 1
        j = j + 1
    sql2 = sql2 + "(1,1,1,1,1,1,1,1,1,1,1,1);"
    #print(sql2)
    connection = pymysql.connect(host='localhost',port=int(3306),user='root',passwd='wikiwiki2',db='Wiki',charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
    with connection:
        with connection.cursor() as cursor:
                cursor.execute(sql2)
        connection.commit()
    end = time.time()
    string2= str((int(i/41326271*100)))+"% i.e. " +str(i) + " in "+str(end-start)
    print(string2 ,end = "\r")

KeyboardInterrupt: 

In [107]:
import requests
r = requests.get(url='https://en.wikipedia.org/w/api.php?action=query&prop=extracts&explaintext&format=json&exsectionformat=wiki&titles=anarchism')
k = r.json()

In [112]:
type( k['query']['pages'][x]['extract'])

str

In [111]:
for x in k['query']['pages']:
    print(x)
    print(k['query']['pages'])
    i = k['query']['pages'][x]['extract'].find('ine')
    j = i
    ch = k['query']['pages'][x]['extract'][i]
    while ch != '.' :
        i -= 1
        ch = k['query']['pages'][x]['extract'][i]
    start = i +1
    ch = k['query']['pages'][x]['extract'][j]
    while ch != '.': 
        j+= 1
        ch = k['query']['pages'][x]['extract'][j]
    st = ""
    while start < j:
        st +=  k['query']['pages'][x]['extract'][start]
        start+= 1
    print(st)

12
{'12': {'pageid': 12, 'ns': 0, 'title': 'Anarchism', 'extract': 'Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy. Anarchism calls for the abolition of the state, which it holds to be undesirable, unnecessary, and harmful. It is usually described alongside libertarian Marxism as the libertarian wing (libertarian socialism) of the socialist movement and as having a historical association with anti-capitalism and socialism.\nThe history of anarchy goes back to prehistory, when humans arguably lived in anarchic societies long before the establishment of formal states, realms or empires. With the rise of organised hierarchical bodies, scepticism toward authority also rose, but it was not until the 19th century that a self-conscious political movement emerged. During the latter half of the 19th and the first decades of the 20th century, the anarchist movement flourished in most parts of the world and 